# 🦜🔗 LangChain agent using plan and solve

In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain.tools import tool
from langchain import LLMMathChain
from langchain.agents import load_tools,initialize_agent, AgentType
from langchain.utilities.jira import JiraAPIWrapper
from langchain.agents.agent_toolkits.jira.toolkit import JiraToolkit
from dotenv import load_dotenv

load_dotenv()
os.environ["JIRA_API_TOKEN"] = "ATATT3xFfGF0rq2dxlYIsEHCsINCq5MArxY7blUI0ZZIfy8jRjQiY7yE0NN7u7ooULXFtsVnqGtUShkwXNW8y4ko7DSr6GJclZaPcdALYpaDP2jp3OtMU1DfZNpfp4SOsvMw32sA8P_Kckvdf5rAFofOsrbEVPgLVa3ePqmeoKRYm-TIYxZnv7Q=37F9E163"
os.environ["JIRA_USERNAME"] = "johann-peter.hartmann@mayflower.de"
os.environ["JIRA_INSTANCE_URL"] = "https://langchaindemo.atlassian.net"

In [ ]:
tools = load_tools(
    ["serpapi", "human"]
)

jira = JiraAPIWrapper()
jiratoolkit = JiraToolkit.from_jira_api_wrapper(jira)
llm = ChatOpenAI(temperature=0)

jiraagent = initialize_agent(
    jiratoolkit.get_tools(), llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)


@tool("save", return_direct=True)
def save(userstories: list[str]) -> str:
    """Saves a list of user story into the backlog."""
    for userstory in userstories:
        jiraagent.run(f"create a new story in project TICTACTOE: {userstory}.")
    return "Done."

tools = tools + [save]
tools

In [ ]:
model = ChatOpenAI(model="gpt-4", temperature=0.0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)


In [ ]:
agent.run("Create a backlog for a tictactoe game. When you are done save the final user stories to the backlog.")